In [ ]:
#clone all the genomes data

!git clone https://github.com/Shinzen31/genomes.git

# -*- coding: utf-8 -*-

########################################
# 1. Environment Setup and Dependency Installation (can be executed in Colab)
########################################
!pip install -q gurobipy cobra modelseedpy pyrodigal biopython

import warnings
from sklearn.exceptions import InconsistentVersionWarning
warnings.filterwarnings("ignore", category=InconsistentVersionWarning)

import os, glob, gzip, shutil, re

# Configure the Gurobi license (please replace with your own WLSACCESSID, WLSSECRET, LICENSEID)
WLSACCESSID = "9cff747c-69bf-432f-8a5e-626093428473"
WLSSECRET   = "94a4f3c2-913b-4777-ad5a-9d9598a0f84c"
LICENSEID   = "2646038"

os.makedirs("/content/licenses", exist_ok=True)
gurobi_lic_text = f"""# Gurobi WLS license file
WLSACCESSID={WLSACCESSID}
WLSSECRET={WLSSECRET}
LICENSEID={LICENSEID}
"""
with open("/content/licenses/gurobi.lic", "w") as f:
    f.write(gurobi_lic_text)
os.environ["GRB_LICENSE_FILE"] = "/content/licenses/gurobi.lic"

import gurobipy
print("Gurobi ready!")

import cobra
try:
    test_m = cobra.io.load_model("textbook")
    sol = test_m.optimize()
    print("COBRApy ok! textbook obj =", sol.objective_value)
except Exception as e:
    print("COBRApy fallback test:", e)


Cloning into 'genomes'...
remote: Enumerating objects: 50, done.
remote: Counting objects: 100% (50/50), done.
remote: Compressing objects: 100% (50/50), done.
remote: Total 50 (delta 3), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (50/50), 21.58 MiB | 30.66 MiB/s, done.
Resolving deltas: 100% (3/3), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.4/56.4 kB 1.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.6/467.6 kB 12.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.4/14.4 MB 60.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 36.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.1/175.1 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 45.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

**Data file format conversion and draft model construction**

In [ ]:
########################################
# 2. fna(.gz) → faa (using Pyrodigal)
########################################
import pyrodigal
from Bio import SeqIO

def convert_fna_to_faa(fna_file, faa_file):
    """
    Perform ORF prediction using the metagenomic mode of Pyrodigal and generate a .faa file
    """
    if fna_file.lower().endswith(".gz"):
        handle = gzip.open(fna_file, "rt")
    else:
        handle = open(fna_file, "r")
    recs = list(SeqIO.parse(handle, "fasta"))
    handle.close()

    gf = pyrodigal.GeneFinder(meta=True)
    with open(faa_file, "w") as fout:
        for rec in recs:
            genes = gf.find_genes(str(rec.seq))
            genes.write_translations(fout, sequence_id=rec.id)

    print(f"[Done] {fna_file} -> {faa_file}")


def gunzip_faa(faa_gz, faa_out):
    """
    Decompress a .faa.gz file to .faa
    """
    with gzip.open(faa_gz, "rt") as zf, open(faa_out, "w") as out_f:
        out_f.write(zf.read())
    print(f"[Decompressed] {faa_gz} -> {faa_out}")


########################################
# 3. Batch Build Draft Models (without ID repair / without adding biomass)
########################################
from modelseedpy import MSBuilder, MSGenome

# Create output directory
models_dir = "/content/models"
os.makedirs(models_dir, exist_ok=True)

genomes_dir = "/content/genomes"
# Collect all input files (fna, fna.gz, faa, faa.gz)
input_files = sorted(glob.glob(os.path.join(genomes_dir, "*.fna"))) + \
              sorted(glob.glob(os.path.join(genomes_dir, "*.fna.gz"))) + \
              sorted(glob.glob(os.path.join(genomes_dir, "*.faa"))) + \
              sorted(glob.glob(os.path.join(genomes_dir, "*.faa.gz")))

print(f"Found {len(input_files)} input genomes in {genomes_dir}.")

for input_file in input_files:
    # Extract model name (remove path and extension, used for model.id and output file name)
    filename = os.path.basename(input_file)
    base_name = filename
    if filename.endswith(".gz"):
        base_name = base_name[:-3]  # Remove .gz
    if base_name.endswith(".fna") or base_name.endswith(".faa"):
        base_name = base_name[:-4]  # Remove .fna or .faa

    # Default path for generated/used .faa file
    faa_path = os.path.join("/content", base_name + ".faa")

    # If the input is fna or fna.gz, perform ORF prediction first
    if input_file.endswith(".fna") or input_file.endswith(".fna.gz"):
        convert_fna_to_faa(input_file, faa_path)
    # If the input is .faa.gz, decompress it to .faa
    elif input_file.endswith(".faa.gz"):
        gunzip_faa(input_file, faa_path)
    else:
        # If it is already .faa, no further processing is required
        faa_path = input_file

    # Use ModelSEEDpy to build the draft model
    ms_genome = MSGenome.from_fasta(faa_path, split=' ')

    try:
        model = MSBuilder.build_metabolic_model(
            model_id=base_name,
            genome=ms_genome,
            index="0",
            gapfill_model=False,
            gapfill_media=None,
            annotate_with_rast=True,           # Note: Requires RAST credentials
            allow_all_non_grp_reactions=True
        )
    except Exception as e:
        print(f"[Error] Model construction failed: {input_file} => {e}")
        continue

    # Do not perform ID repair, and do not add the biomass (bio1) reaction
    # model.id = base_name  # Only set model.id to base_name

    # Save the model
    sbml_path = os.path.join(models_dir, base_name + ".xml")
    cobra.io.write_sbml_model(model, sbml_path)


print("\nAll draft models built.")


modelseedpy 0.4.2
Found 40 input genomes in /content/genomes.
[Done] /content/genomes/GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic.fna.gz -> /content/GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic.faa


<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()


[Done] /content/genomes/GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic.fna.gz -> /content/GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic.faa


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[Done] /content/genomes/GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic.fna.gz -> /content/GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic.faa


...

[Decompressed] /content/genomes/GCF_964084875.1_Leuconostoc_carnosum_A1b-VegA_protein.faa.gz -> /content/GCF_964084875.1_Leuconostoc_carnosum_A1b-VegA_protein.faa


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[Decompressed] /content/genomes/GCF_964084915.1_Lactiplantibacillus_plantarum_ATCC14431_protein.faa.gz -> /content/GCF_964084915.1_Lactiplantibacillus_plantarum_ATCC14431_protein.faa


/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator KNeighborsClassifier from version 1.2.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(



All draft models built.


**Draft model infomation print（Facilitates the identification of oxygen-related metabolites.）**

In [ ]:
import os
import cobra

models_dir = "/content/models"  # The folder contains 39 models
model_files = sorted([f for f in os.listdir(models_dir) if f.endswith(".xml")])

for filename in model_files:
    model_path = os.path.join(models_dir, filename)
    try:
        model = cobra.io.read_sbml_model(model_path)
    except Exception as e:
        print(f"Error loading model {filename}: {e}")
        continue

    print(f"\n------ Model: {filename} (ID: {model.id}) ------")
    for metabolite in model.metabolites:
        reacStr = metabolite.name.lower()
        if 'o2' in reacStr:
            print(metabolite.name, reacStr, metabolite.id)
        if 'oxygen' in reacStr:
            print(metabolite.name, reacStr, metabolite.id)


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
O2 [c0] o2 [c0] cpd00007_c0
H2O2 [c0] h2o2 [c0] cpd00025_c0
O2- [c0] o2- [c0] cpd00532_c0
Co2+ [c0] co2+ [c0] cpd00149_c0
Co2+ [e0] co2+ [e0] cpd00149_e0
O2 [e0] o2 [e0] cpd00007_e0
CO2 [e0] co2 [e0] cpd00011_e0

------ Model: GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
O2 [c0] o2 [c0] cpd00007_c0
H2O2 [c0] h2o2 [c0] cpd00025_c0
kdo2-lipid a [c0] kdo2-lipid a [c0] cpd03587_c0
heptosyl-kdo2-lipidA [c0] heptosyl-kdo2-lipida [c0] cpd15485_c0
kdo-phospho-heptosyl-phospho-heptosyl-heptosyl-kdo2-lipidA [c0] kdo-phospho-heptosyl-phospho-heptosyl-heptosyl-kdo2-lipida [c0] cpd15492_c0
phospho-heptosyl-heptosyl-kdo2-lipidA [c0] phospho-heptosyl-heptosyl-kdo2-lipida [c0] cpd15549_c0
heptosyl-phospho-heptosyl-heptosyl-kdo2-lipidA [c0] heptosyl-phospho-heptosyl-heptosyl-kdo2-lipida [c0] cpd15488_c0
O2- [c0] o2- [c0] cpd00

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
O2 [c0] o2 [c0] cpd00007_c0
H2O2 [c0] h2o2 [c0] cpd00025_c0
Co2+ [c0] co2+ [c0] cpd00149_c0
Co2+ [e0] co2+ [e0] cpd00149_e0
O2 [e0] o2 [e0] cpd00007_e0
CO2 [e0] co2 [e0] cpd00011_e0

------ Model: GCF_000010425.1_ASM1042v1_protein.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
Co2+ [e0] co2+ [e0] cpd00149_e0
Co2+ [c0] co2+ [c0] cpd00149_c0
O2 [e0] o2 [e0] cpd00007_e0
O2 [c0] o2 [c0] cpd00007_c0
CO2 [e0] co2 [e0] cpd00011_e0
H2O2 [c0] h2o2 [c0] cpd00025_c0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


...


------ Model: GCF_964069135.1_Schleiferilactobacillus_harbinensis_CIRM-BIA2388_protein.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
O2 [c0] o2 [c0] cpd00007_c0
H2O2 [c0] h2o2 [c0] cpd00025_c0
Co2+ [c0] co2+ [c0] cpd00149_c0
Co2+ [e0] co2+ [e0] cpd00149_e0
O2 [e0] o2 [e0] cpd00007_e0
CO2 [e0] co2 [e0] cpd00011_e0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCF_964084875.1_Leuconostoc_carnosum_A1b-VegA_protein.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
O2 [c0] o2 [c0] cpd00007_c0
H2O2 [c0] h2o2 [c0] cpd00025_c0
Co2+ [c0] co2+ [c0] cpd00149_c0
Co2+ [e0] co2+ [e0] cpd00149_e0
O2 [e0] o2 [e0] cpd00007_e0
CO2 [e0] co2 [e0] cpd00011_e0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCF_964084915.1_Lactiplantibacillus_plantarum_ATCC14431_protein.xml (ID: ) ------
CO2 [c0] co2 [c0] cpd00011_c0
O2 [c0] o2 [c0] cpd00007_c0
H2O2 [c0] h2o2 [c0] cpd00025_c0
Co2+ [c0] co2+ [c0] cpd00149_c0
Co2+ [e0] co2+ [e0] cpd00149_e0
O2 [e0] o2 [e0] cpd00007_e0
CO2 [e0] co2 [e0] cpd00011_e0


**Construct a complete medium and print the list of oxygen-related reactions for each draft model.**

In [ ]:
import os
import cobra
import re

# Define your model directory that contains 39 draft models (*.xml)
models_dir = "/content/models"
model_files = sorted([f for f in os.listdir(models_dir) if f.endswith(".xml")])

# Define a blacklist of keywords related to oxygen (these keywords only match as independent words)
black_list = [
    "o2",
    "o2-",
    "oxygen",
    "superoxide",
    "peroxide"
]

# Modified helper function: use regular expressions to ensure only matching independent keywords (word boundaries)
def is_oxygen_related(met_name, blacklist):
    lower_name = met_name.lower()
    for term in blacklist:
        # Use \b word boundary to match independent keywords
        if re.search(rf"\b{re.escape(term)}\b", lower_name):
            return True
    return False

# Traverse all models and merge all metabolites into a dictionary; the key is the cleaned-up ID and the value is the metabolite name.
all_mets = {}
for mf in model_files:
    model_path = os.path.join(models_dir, mf)
    try:
        model = cobra.io.read_sbml_model(model_path)
    except Exception as e:
        print(f"[Error] Failed to load model {mf}: {e}")
        continue

    # Traverse all metabolites in the current model
    for met in model.metabolites:
        # If the metabolite name contains oxygen-related keywords, skip it
        if met.compartment=='c0':
          continue
        if is_oxygen_related(met.name, black_list):
            continue
        # Remove potential "_e0" and "_c0" suffixes to get a cleaner ID
        tidy_id = met.id.replace("_e0", "").replace("_c0", "")
        # If this tidy_id is not already added, then add it to the dictionary
        if tidy_id not in all_mets:
            all_mets[tidy_id] = met.name

# Write the merged metabolites dictionary into the oxygen-free medium file
tsv_path = "complete_media_without_oxygen.tsv"
with open(tsv_path, "w") as f:
    f.write("id\tname\tconcentration\tminflux\tmaxflux\n")
    for met_id, met_name in all_mets.items():
        f.write(f"{met_id}\t{met_name}\t1\t-1000\t1000\n")

print(f"\n[Oxygen-free medium file saved] => {tsv_path}")

# ---------------------------------------------
# Now traverse all models and print the oxygen-related metabolite information,
# so that you can adjust the blacklist later if needed.
print("\n=== Checking oxygen-related metabolites in all 39 models ===")
for mf in model_files:
    model_path = os.path.join(models_dir, mf)
    try:
        model = cobra.io.read_sbml_model(model_path)
    except Exception as e:
        print(f"[Error] Failed to load model {mf}: {e}")
        continue
    print(f"\n------ Model: {mf} (ID: {model.id if model.id else 'N/A'}) ------")
    for met in model.metabolites:
        if is_oxygen_related(met.name, black_list):
            print(f"{met.name} [{met.compartment}]  => {met.id}")

print("\n[Check completed]")


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



[Oxygen-free medium file saved] => complete_media_without_oxygen.tsv

=== Checking oxygen-related metabolites in all 39 models ===

------ Model: GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic.xml (ID: N/A) ------
O2 [c0] [c0]  => cpd00007_c0
O2- [c0] [c0]  => cpd00532_c0
O2 [e0] [e0]  => cpd00007_e0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic.xml (ID: N/A) ------
O2 [c0] [c0]  => cpd00007_c0
O2- [c0] [c0]  => cpd00532_c0
O2 [e0] [e0]  => cpd00007_e0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


...


------ Model: GCF_964068245.1_Lentilactobacillus_kefiri_CIRM-BIA2539_protein.xml (ID: N/A) ------
O2 [c0] [c0]  => cpd00007_c0
O2 [e0] [e0]  => cpd00007_e0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCF_964069135.1_Schleiferilactobacillus_harbinensis_CIRM-BIA2388_protein.xml (ID: N/A) ------
O2 [c0] [c0]  => cpd00007_c0
O2 [e0] [e0]  => cpd00007_e0

------ Model: GCF_964084875.1_Leuconostoc_carnosum_A1b-VegA_protein.xml (ID: N/A) ------
O2 [c0] [c0]  => cpd00007_c0
O2 [e0] [e0]  => cpd00007_e0


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.



------ Model: GCF_964084915.1_Lactiplantibacillus_plantarum_ATCC14431_protein.xml (ID: N/A) ------
O2 [c0] [c0]  => cpd00007_c0
O2 [e0] [e0]  => cpd00007_e0

[Check completed]


In [ ]:
!cat complete_media_without_oxygen.tsv

id	name	concentration	minflux	maxflux
cpd00067	H+ [e0]	1	-1000	1000
cpd00075	Nitrite [e0]	1	-1000	1000
cpd11576	L-methionine R-oxide [e0]	1	-1000	1000
cpd04097	Pb [e0]	1	-1000	1000
cpd01030	Salicin [e0]	1	-1000	1000
cpd01012	Cd2+ [e0]	1	-1000	1000
cpd00276	GLUM [e0]	1	-1000	1000
cpd00082	D-Fructose [e0]	1	-1000	1000
cpd00058	Cu2+ [e0]	1	-1000	1000
cpd00080	Glycerol-3-phosphate [e0]	1	-1000	1000
cpd00314	D-Mannitol [e0]	1	-1000	1000
cpd00222	GLCN [e0]	1	-1000	1000
cpd00013	NH3 [e0]	1	-1000	1000
cpd00051	L-Arginine [e0]	1	-1000	1000
cpd00064	Ornithine [e0]	1	-1000	1000
cpd00244	Ni2+ [e0]	1	-1000	1000
cpd00179	Maltose [e0]	1	-1000	1000
cpd00205	K+ [e0]	1	-1000	1000
cpd00355	Nicotinamide ribonucleotide [e0]	1	-1000	1000
cpd00100	Glycerol [e0]	1	-1000	1000
cpd00009	Phosphate [e0]	1	-1000	1000
cpd00060	L-Methionine [e0]	1	-1000	1000
cpd00137	Citrate [e0]	1	-1000	1000
cpd00034	Zn2+ [e0]	1	-1000	1000
cpd00076	Sucrose [e0]	1	-1000	1000
cpd01171	Dulcose [e0]	1	-1000	1000
cpd00023	L-Glutamate [e0

In [ ]:
# Define the list of IDs to check
to_check = ["cpd00007_c0", "cpd00532_c0", "cpd00001"]

# Initialize a dictionary to record whether each ID is found
found = {key: False for key in to_check}

# Open and read the "complete_media_without_oxygen.tsv" file
tsv_path = "complete_media_without_oxygen.tsv"

with open(tsv_path, "r") as f:
    # Read all lines while skipping the header
    lines = f.readlines()[1:]
    for line in lines:
        # Split each line into fields based on the tab delimiter
        parts = line.strip().split("\t")
        if parts:
            id_val = parts[0]
            if id_val in to_check:
                found[id_val] = True

# Output the checking results
for key in to_check:
    print(f"{key}: {'Found' if found[key] else 'Not Found'}")


cpd00007_c0: Not Found
cpd00532_c0: Not Found
cpd00001: Found


In [ ]:
from cobra import Reaction

def makeExchangeReaction(model, met):
  new_reaction = Reaction('EX_' + met.id)
  new_reaction.name = met.name + 'exchange'
  new_reaction.lower_bound = -1000
  new_reaction.upper_bound = 1000
  new_reaction.add_metabolites({
    met: -1.0})

  model.add_reactions([new_reaction])
  return model

def fix_exchange_reactions(model_in):
  for met in model_in.metabolites:
    if met.compartment == 'e0':
      if not model_in.reactions.has_id('EX_' + met.id):
        model_in = makeExchangeReaction(model_in, met)
  return model_in.copy()

# Gapfilling

In [ ]:
# BLOCK X: Gapfilling all 39 models with "complete_media_without_oxygen.tsv"

# 1) First, view the first few lines of the medium file to ensure the oxygen-free medium file is correct
!head complete_media_without_oxygen.tsv

# 2) List the files in the current directory to check if your draft models folder exists, etc.
!ls
!pip install dnngior --no-deps

import os
import cobra
import dnngior
import traceback

# Define input and output directories
models_dir = "/content/models"  # Folder containing 39 draft model (*.xml) files
output_dir = "/content/models_gapfilled"  # Folder to store the gapfilled models
os.makedirs(output_dir, exist_ok=True)

# Collect all model files ending with .xml
model_files = sorted([f for f in os.listdir(models_dir) if f.endswith(".xml")])

# Iterate over each model and perform gapfilling
for mf in model_files:
    model_path = os.path.join(models_dir, mf)
    prefix = mf.replace(".xml", "")
    print(f"\n=== Gapfilling {prefix} with no-oxygen medium ===")

    try:
        # Use DNNGIOR to perform gapfilling
        gapfill_no_oxygen = dnngior.Gapfill(
            model_path,
            medium_file='complete_media_without_oxygen.tsv',
            objectiveName='bio1'
        )

        # gapfilledModel contains the model after gapfilling
        model_gapfilled_no_oxygen = gapfill_no_oxygen.gapfilledModel.copy()

        model_gapfilled_no_oxygen = fix_exchange_reactions(model_gapfilled_no_oxygen)

        # -------- New step: Manually ensure that the bio1 reaction is set as the objective function --------
        try:
            bio_rxn = model_gapfilled_no_oxygen.reactions.get_by_id("bio1")
            bio_rxn.objective_coefficient = 1.0
            model_gapfilled_no_oxygen.objective = bio_rxn
            print("  [Info] 'bio1' objective function has been set with objective_coefficient = 1.0")
        except Exception as e:
            print("  [Warning] 'bio1' not found or unable to set objective_coefficient:", e)

        # Save the gapfilled model to the output folder
        out_path = os.path.join(output_dir, f"{prefix}_gapfilled_noO2.xml")
        cobra.io.write_sbml_model(model_gapfilled_no_oxygen, out_path)

        # Print the summary of the objective function to check if biomass can be produced after gapfilling
        print("Objective function (bio1) summary:")
        # First, check the optimization result before calling summary
        solution = model_gapfilled_no_oxygen.optimize()
        print("  FBA solution objective:", solution.objective_value)
        # Optionally, use model_gapfilled_no_oxygen.summary() to see a detailed flux distribution
        print(model_gapfilled_no_oxygen.summary())

        print(f"[Saved] Gapfilled model => {out_path}\n")

    except Exception as e:
        print(f"[Error] Gapfilling failed for {mf}: {e}")
        traceback.print_exc()

print("\n[All gapfilling done! Models saved in /content/models_gapfilled.]")


id	name	concentration	minflux	maxflux
cpd00067	H+ [e0]	1	-1000	1000
cpd00075	Nitrite [e0]	1	-1000	1000
cpd11576	L-methionine R-oxide [e0]	1	-1000	1000
cpd04097	Pb [e0]	1	-1000	1000
cpd01030	Salicin [e0]	1	-1000	1000
cpd01012	Cd2+ [e0]	1	-1000	1000
cpd00276	GLUM [e0]	1	-1000	1000
cpd00082	D-Fructose [e0]	1	-1000	1000
cpd00058	Cu2+ [e0]	1	-1000	1000
complete_media_without_oxygen.tsv
GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic.faa
GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic.faa
GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic.faa
GCF_000010425.1_ASM1042v1_protein.faa
GCF_964060345.1_Lentilactobacillus_buchneri_CIRM-BIA1514_protein.faa
GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_protein.faa
GCF_964062605.1_Lentilactobacillus_kefiri_CIRM-BIA2556_protein.faa
GCF_964062635.1_Pediococcus_ethanolidurans_CIRM-BIA2669_protein.faa
GCF_964062735.1_Lentilactobacillus_parabuchneri_CIRM-BIA2637_protein.faa
GCF_964062745.1_Levilactobacillus_cerevisiae_CIRM-BIA2671_

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
/usr/local/lib/python3.11/dist-packages/dnngior/addExchanges.py:18: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/dnngior/files/biochemistry/compounds.tsv' mode='r' encoding='utf-8'>
  compounds_helper = Compounds()
/usr/local/lib/python3.11/dist-packages/dnngior/addExchanges.py:19: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/dnngior/files/biochemistry/compounds.tsv' mode='r' encoding='utf-8'>
  compounds_dict   = compounds_helper.loadCompounds()
/usr/local/lib/python3.11/dist-packages/dnngior/build_model.py:13: ResourceWarning: unclosed file <_io.TextIOWrapper name='/usr/local/lib/python3.11/dist-packages/dnngior/files/biochemistry/compounds.tsv

The rest of dnngior features can be used without it.

=== Gapfilling GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic with no-oxygen medium ===


ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Gap-filling database =  ModelSEED


...

Loading medium from: complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  6 / 672
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  77047 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  38523 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19261 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9630 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4815 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2407 


Warning for adding constraints: zero or small (< 1

ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.
ERROR:cobra.io.sbml:'' is not a valid SBML 'SId'.


Gap-filling database =  ModelSEED


ERROR:cobra.io.sbml:No objective coefficients in model. Unclear what should be optimized


Loading medium from: complete_media_without_oxygen.tsv
#reactions not found in NN-keys:  8 / 812
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Flux through biomass reaction is 1.00000000
Flux through biomass reaction is 1.00000000
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  76792 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  38396 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  19198 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  9599 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  4799 


Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored


 condition is currently:  2399 


Warning for adding constraints: zero or small (< 1

**Extract & Unify Exchange Reactions from All Gapfilled Models**

In [ ]:
import os
import cobra
import numpy as np

gapfilled_dir = "/content/models_gapfilled"  # folder containing 39 gapfilled models
model_files = sorted([f for f in os.listdir(gapfilled_dir) if f.endswith(".xml")])

def extract_exchange_reactions(model):
    """
    Return a dictionary of exchange reactions from the model
    that have negative lower_bound (< 0), i.e., import direction.

    Format: {rxn_id: [met_id, met_name, met_formula]}
    """
    exchange_reactions = {}
    for reaction in model.reactions:
        # Common practice: exchange rxns often start with "EX_",
        # but you can adapt this filter if your exchange IDs differ
        if reaction.id.startswith("EX_") and (reaction.lower_bound < 0):
            # Typically there's only 1 metabolite in an exchange reaction,
            # so we grab the first one
            if len(reaction.metabolites) == 1:
                met = list(reaction.metabolites.keys())[0]
                exchange_reactions[reaction.id] = [met.id, met.name, met.formula]

    return exchange_reactions

# 1) Collect exchange reactions from each gapfilled model
all_exchange_dict = {}  # key=rxn_id, value=[met_id, met_name, met_formula]

for mf in model_files:
    model_path = os.path.join(gapfilled_dir, mf)
    print(f"Extracting exchange rxns from {mf}...")
    try:
        model = cobra.io.read_sbml_model(model_path)
    except Exception as e:
        print(f"[Error] Cannot load {mf}: {e}")
        continue

    ex_rxns = extract_exchange_reactions(model)

    # Unify them
    for rxn_id, info_list in ex_rxns.items():
        if rxn_id not in all_exchange_dict:
            all_exchange_dict[rxn_id] = info_list

# 2) Write the unified exchange reactions to a file
medium_file = "group_medium_exchanges.tsv"
with open(medium_file, "w") as f:
    f.write("rxn_id\tmet_id\tmet_name\tmet_formula\n")
    for rxn_id, info in all_exchange_dict.items():
        met_id, met_name, met_form = info
        f.write(f"{rxn_id}\t{met_id}\t{met_name}\t{met_form}\n")

print(f"\n[Unified exchange reactions saved to: {medium_file}]")
print(f"Total unique exchange reactions: {len(all_exchange_dict)}")


Extracting exchange rxns from GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCF_000010425.1_ASM1042v1_protein_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCF_964060345.1_Lentilactobacillus_buchneri_CIRM-BIA1514_protein_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCF_964062525.1_Enterococcus_gilvus_CIRM-BIA2700_protein_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCF_964062605.1_Lentilactobacillus_kefiri_CIRM-BIA2556_protein_gapfilled_noO2_appliedMedium.xml...
Extracting exchange rxns from GCF_964062635.1_Pediococcus_ethanolidurans_CIRM-BIA2669_protein_gapfilled_noO2_appliedMedium.xml...
Extracting exchange

**Apply the Unified Exchange List to All Gapfilled Models**

In [ ]:
import random

def apply_medium(model, medium_dict, optimize=True):
    """
    Given a dictionary of {rxn_id: flux_value}, set the reaction lower_bound in the model.
    E.g., if flux_value = 5 then reaction.lower_bound = -5 (for import direction).
    """
    for rxn_id, flux_value in medium_dict.items():
        if rxn_id in model.reactions:
            rxn = model.reactions.get_by_id(rxn_id)
            rxn.lower_bound = -flux_value
    if optimize:
        sol = model.optimize()
        return sol
    return None

# 1) Read the unified exchange list from the file
ex_file = "group_medium_exchanges.tsv"  # Note: File name has been changed to "group_medium_exchanges.tsv" if needed
exchange_list = []
with open(ex_file, "r") as f:
    lines = f.read().strip().split("\n")[1:]  # Skip header
    for line in lines:
        rxn_id, met_id, met_name, met_formula = line.split("\t")
        exchange_list.append(rxn_id)

# 2) Build a dictionary of flux values for each exchange.
# For demonstration, we pick random flux bounds in the range [0, 10] for each reaction.
medium_dict = {}
for rxn_id in exchange_list:
    medium_dict[rxn_id] = random.uniform(0, 10)

# 3) Apply this "medium" to each gapfilled model.
for mf in model_files:
    model_path = os.path.join(gapfilled_dir, mf)
    prefix = mf.replace(".xml", "")
    print(f"\nApplying medium to {prefix}...")

    try:
        model = cobra.io.read_sbml_model(model_path)
    except Exception as e:
        print(f"[Error] Loading {mf}: {e}")
        continue

    # Apply the new exchange flux bounds
    solution = apply_medium(model, medium_dict, optimize=True)
    if solution:
        print(f"  After applying medium, objective = {solution.objective_value}")
    else:
        print("  No solution returned.")

    # (Optional) Save the updated model if desired
    out_path = os.path.join(gapfilled_dir, f"{prefix}_appliedMedium.xml")
    cobra.io.write_sbml_model(model, out_path)
    print(f"  => Updated model saved: {out_path}")



Applying medium to GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic_gapfilled_noO2_appliedMedium...
  After applying medium, objective = 12.175685452659103
  => Updated model saved: /content/models_gapfilled/GCA_964062575.1_Lactococcus_lactis_CIRM-BIA2698_genomic_gapfilled_noO2_appliedMedium_appliedMedium.xml

Applying medium to GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic_gapfilled_noO2_appliedMedium...
  After applying medium, objective = 13.977121869796349
  => Updated model saved: /content/models_gapfilled/GCA_964063365.1_Rahnella_aquatiliss_CIRM-BIA2795_genomic_gapfilled_noO2_appliedMedium_appliedMedium.xml

Applying medium to GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic_gapfilled_noO2_appliedMedium...
  After applying medium, objective = 12.175685452659103
  => Updated model saved: /content/models_gapfilled/GCA_964063435.1_Leuconostoc_mesenteroides_FME302_genomic_gapfilled_noO2_appliedMedium_appliedMedium.xml

Applying medium to GCF_000010425.1_ASM

In [ ]:
!zip -r /content/content.zip /content
from google.colab import files
files.download("/content/content.zip")

  adding: content/ (stored 0%)
  adding: content/.config/ (stored 0%)
  adding: content/.config/logs/ (stored 0%)
  adding: content/.config/logs/2025.04.21/ (stored 0%)
  adding: content/.config/logs/2025.04.21/18.42.22.411154.log (deflated 58%)
  adding: content/.config/logs/2025.04.21/18.42.31.744939.log (deflated 56%)
  adding: content/.config/logs/2025.04.21/18.42.12.824329.log (deflated 58%)
  adding: content/.config/logs/2025.04.21/18.42.21.235934.log (deflated 86%)
  adding: content/.config/logs/2025.04.21/18.42.31.045496.log (deflated 57%)
  adding: content/.config/logs/2025.04.21/18.41.51.884422.log (deflated 92%)
  adding: content/.config/configurations/ (stored 0%)
  adding: content/.config/configurations/config_default (deflated 15%)
  adding: content/.config/default_configs.db (deflated 98%)
  adding: content/.config/.last_opt_in_prompt.yaml (stored 0%)
  adding: content/.config/active_config (stored 0%)
  adding: content/.config/gce (stored 0%)
  adding: content/.config/.

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<frozen importlib._bootstrap>:1047: ImportWarning: _PyDriveImportHook.find_spec() not found; falling back to find_module()
<frozen importlib._bootstrap>:1047: ImportWarning: _BokehImportHook.find_spec() not found; falling back to find_module()
